In [ ]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
import statsmodels.api as sm
import numpy as np
from itertools import product

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]


In [ ]:
zaps = dt.get_zaps()
posts = dt.get_posts()


In [ ]:
posts